In [1]:
# Subject:      NLP - gleaning topics from 2016 presidential debate transcripts 
# Date:         11/14/2019
# Name:         Sami Ahmed
# Worked with:  (solo, mostly consulted with John) 


**notebook objective: Cleaning corpus, initial LSA topic modeling**

## imports/load data

In [2]:
# NLP related packages
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


#misc files/text processing
import pickle
import re
import collections
import pprint

#  pandas  options

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# import sklearn for Kmeans
from sklearn.preprocessing import scale
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs
from sklearn.datasets import fetch_mldata
from sklearn.utils import shuffle


In [21]:
# load debate data from mongo database (reference the mongo_project4 notebook) 

with open('twenty16_debate_corpus.pkl', 'rb') as f:
       twenty16_debate_corpus = pickle.load(f)

## EDA 

In [23]:
# convert dict to data frame, drop cols I will definitely not use 

data_df = pd.DataFrame.from_dict(twenty16_debate_corpus)
data_df = data_df.drop(['URL', '_id'], axis = 1)

In [24]:
data_df.columns

Index(['Speaker', 'Text', 'Date', 'Party', 'Location'], dtype='object')

In [19]:
# pulling out all the candidates to add to stop words to corpus

Speakers = data_df['Speaker'].unique()
Speakers = [x.lower() for x in Speakers]

In [6]:
# isolate to just the text of the dataframe

only_text = data_df['Text']
print(only_text.shape)
only_text[1:50]

(13254,)


1     We want to also extend our warm thanks to Milw...
2                                            (APPLAUSE)
3     Welcome, Senator, great to see you. And former...
4                                            (APPLAUSE)
5                        Very good to be here with you.
6                                            Thank you.
7                                  Welcome to you both.
8     Now, a word about format. There will be two sh...
9     With Iowa and New Hampshire behind us, we are ...
10    Well, Gwen and Judy, thank you very much for h...
11                          Thank you, Senator Sanders.
12                                           (APPLAUSE)
13       Thank you, Senator Sanders. Secretary Clinton.
14    I'm running for president to knock down all th...
15                                      Thank you both.
16                                           (APPLAUSE)
17    Thank you both. And we'll be right back after ...
18                                         (COMM

## Clean corups function  

In [13]:
import re
import string

def clean_text_round1(text):
    '''make text lowercase, remove text in parantheses, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\(.*?\)', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [25]:
# evaluate cleaned up text 

data_df['Text'] = pd.DataFrame(data_df['Text'].apply(round1))

In [ ]:
# pickle cleaned data, I continue using this data in the LDA_model_&_lemmatization notebook

data_clean.to_pickle("data_clean_debate.pkl")

**after running the clean_text function above, I am going to try running LSA just to see how it does without further preprocessing (i.e. lemmitization etc.)**

## Words to Vectors - unigrams

In [33]:
# make dict of stop words, add common debate words that came up in initial topics out of LSA 

from sklearn.feature_extraction import text 

stop_words = text.ENGLISH_STOP_WORDS.union(['im','dont','need','want','senator','governor','know',
                                           'come','theyre','youre','going','think','said','thats',
                                           'just','make','did','got','mr','ms','ive'])
debate_list = stop_words

In [276]:
# This first document-term matrix has default Count Vectorizer values - counts of unigrams

vectorizer = CountVectorizer(stop_words=debate_list,min_df=10, max_df=8.5)
doc_word = vectorizer.fit_transform(data_clean['Text']) # get all of the counts
doc_word.shape

(13254, 2583)

In [278]:
# pickle countvertorizer and words after document-term matrix format 

data_dtm.to_pickle('dtm.pkl')
pickle.dump(vectorizer, open("vectorizer.pkl", "wb"))

In [277]:
# We have to convert `.toarray()` because the vectorizer returns a sparse matrix.

data_dtm = pd.DataFrame(doc_word.toarray(), index=data_clean, columns=vectorizer.get_feature_names())

## Initial LSA model

In [207]:
# Acronynms: Latent Semantic Analysis (LSA) is just another name for 
#  Signular Value Decomposition (SVD) applied to Natural Language Processing (NLP)
lsa = TruncatedSVD(10)
doc_topic = lsa.fit_transform(doc_word)
lsa.explained_variance_ratio_ 

# this tells us what amount of the variance is being captured for topics
# 1 & 2 out of LSA, this is typically low
# do not pay too much attention to these numbers

array([0.04915205, 0.01567465, 0.01403383, 0.00967929, 0.00920427,
       0.0081931 , 0.00774491, 0.00766894, 0.0070081 , 0.00693809])

In [235]:
topic_word = pd.DataFrame(lsa.components_.round(10),
             index = ["component_1","component_2","component_3","component_4","component_5",
                     "component_6", "component_7","component_8","component_9","component_10"],
             columns = vectorizer.get_feature_names())
# the U matrix tells you how the topics are in your data

abc   ability      able   abolish  abortion    abroad  \
component_1   0.000621  0.008889  0.044365  0.001953  0.005166  0.003731   
component_2  -0.000490 -0.002020  0.008148 -0.001383 -0.008467 -0.001219   
component_3  -0.000226 -0.002909 -0.004783  0.007853 -0.001367  0.000356   
component_4  -0.001178 -0.001902 -0.010695  0.001594 -0.011720  0.001081   
component_5  -0.002248  0.001395  0.016193  0.006268 -0.011569  0.002199   
component_6   0.002210 -0.005103 -0.024111  0.002506  0.007389 -0.003474   
component_7  -0.000617  0.003011  0.019152 -0.004681  0.003271  0.000167   
component_8  -0.002117  0.005100 -0.000569 -0.001290 -0.003516 -0.000197   
component_9  -0.003289  0.001027 -0.010917 -0.003935 -0.011722 -0.000307   
component_10  0.004214  0.003048 -0.014882 -0.002004  0.000941  0.002342   

              absolute  absolutely    absurd     abuse    accept  acceptable  \
component_1   0.002099    0.024146  0.001196  0.003733  0.003661    0.002624   
component_2   0.000037   -0.004404 -0.000136 -0.001247 -0.002035   -0.000279   
component_3  -0.000356   -0.003610  0.000200  0.001126 -0.000424    0.001254   
component_4   0.001460   -0.015814 -0.001605 -0.001346  0.001315   -0.000501   
component_5  -0.002715   -0.004780  0.000230 -0.001352 -0.004876    0.000411   
component_6  -0.002034   -0.013710 -0.000930 -0.002411 -0.000176   -0.003471   
component_7   0.002219    0.001689  0.001043  0.003977 -0.002277   -0.002023   
component_8  -0.001639    0.009654 -0.000600 -0.001242  0.002190   -0.000800   
component_9  -0.002874    0.010625  0.000683 -0.000983  0.002483   -0.001789   
component_10 -0.002550   -0.018206 -0.000639 -0.002464 -0.001551   -0.001288   

                access  accomplish  accomplished  accomplishment  according  \
component_1   0.004870    0.002578      0.002008        0.002820   0.003721   
component_2   0.004523    0.000677     -0.000632       -0.001951  -0.003158   
component_3  -0.000515   -0.000693      0.001012        0.000753   0.001955   
component_4  -0.000273   -0.000626      0.000888       -0.007495  -0.001622   
component_5  -0.001736   -0.000118     -0.001315       -0.003973   0.000215   
component_6   0.001883    0.001325      0.001786        0.008763   0.002291   
component_7   0.002603    0.000955      0.000171        0.001746  -0.002810   
component_8   0.016509    0.000879      0.004786        0.005683  -0.004091   
component_9   0.000054    0.001193     -0.001690        0.001283   0.002025   
component_10  0.002576   -0.001845     -0.000034        0.001072   0.002912   

               account  accountability  accountable  accounts  accurate  \
component_1   0.003344        0.002596     0.006255  0.003036  0.001730   
component_2   0.002133       -0.001592     0.000633  0.000244 -0.000018   
component_3   0.003361       -0.001180    -0.002036  0.002907  0.000675   
component_4  -0.005495       -0.000993    -0.009165 -0.006601 -0.002539   
component_5  -0.004017        0.003425     0.004706 -0.001714  0.002172   
component_6   0.007562        0.002851     0.002295  0.003356  0.001688   
component_7   0.004839        0.006070     0.006410  0.009007 -0.000770   
component_8   0.028317        0.001822    -0.002121  0.008702  0.003034   
component_9  -0.006636        0.002041     0.001112 -0.003579 -0.001618   
component_10 -0.002759       -0.000557    -0.000512 -0.001001 -0.001223   

               accused   achieve  acknowledge       act     acted    acting  \
component_1   0.000837  0.002455     0.002121  0.023077  0.001897  0.001977   
component_2  -0.000217 -0.001318    -0.002299 -0.003622 -0.003103 -0.000588   
component_3  -0.001116 -0.001751    -0.001210 -0.009796 -0.000725  0.000140   
component_4  -0.002098  0.001794    -0.001205 -0.019006 -0.001115 -0.000790   
component_5   0.000967 -0.000218    -0.001036 -0.016738  0.000755  0.001117   
component_6  -0.000948  0.001588     0.001772  0.036152  0.000250  0.000103   
component_7  -0.001220  0.001476    -0.000948  0.025

In [237]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

display_topics(lsa, vectorizer.get_feature_names(), 10)


Topic  0
people, country, president, america, say, right, american, states, years, government

Topic  1
people, health, care, million, immigration, pay, young, jobs, insurance, social

Topic  2
tax, percent, jobs, taxes, government, plan, pay, money, economy, cut

Topic  3
country, president, america, tax, jobs, united, world, states, percent, trade

Topic  4
president, tax, people, states, united, isis, obama, taxes, war, plan

Topic  5
president, clinton, secretary, health, thank, care, hillary, people, tax, government

Topic  6
government, president, federal, did, law, state, things, years, washington, time

Topic  7
care, health, states, insurance, united, isis, affordable, world, act, money

Topic  8
government, clinton, america, secretary, world, hillary, economy, big, american, street

Topic  9
states, thank, united, america, new, debate, republican, state, world, trump


## Words to Vectors - Bigrams

In [266]:
# The second document-term matrix has both unigrams and bigrams, and indicators instead of counts

vectorizer2 = CountVectorizer(ngram_range=(1,2), binary=True, stop_words=debate_list)
bi_doc_word = vectorizer2.fit_transform(data_clean['Text']) 
bi_doc_word.shape

(13254, 142672)

In [ ]:
# We have to convert `.toarray()` because the vectorizer returns a sparse matrix.

dtm2 = pd.DataFrame(bi_doc_word.toarray(), index=data_clean, columns=vectorizer2.get_feature_names())

## LSA model - Bigrams

In [270]:
# Acronynms: Latent Semantic Analysis (LSA) is just another name for 
#  Signular Value Decomposition (SVD) applied to Natural Language Processing (NLP)
bi_lsa = TruncatedSVD(10)
bi_doc_topic = bi_lsa.fit_transform(bi_doc_word)
lsa.explained_variance_ratio_ 

# this tells us what amount of the variance is being captured for topics
# 1 & 2 out of LSA, this is typically low
# do not pay too much attention to these numbers

array([0.02074058, 0.00584914, 0.00462338, 0.00401759, 0.00378084,
       0.00366545, 0.00316216, 0.00309986, 0.00295398, 0.0029304 ])

In [271]:
bi_topic_word = pd.DataFrame(bi_lsa.components_.round(10),
             index = ["component_1","component_2","component_3","component_4","component_5",
                     "component_6", "component_7","component_8","component_9","component_10"],
             columns = vectorizer2.get_feature_names())
# the U matrix tells you how the topics are in your data

In [272]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

display_topics(bi_lsa, vectorizer2.get_feature_names(), 10)


Topic  0
people, country, president, say, right, america, states, like, years, way

Topic  1
united, states, united states, president, clinton, president united, obama, secretary, thank, secretary clinton

Topic  2
clinton, secretary, secretary clinton, thank, sanders, hillary, trump, hillary clinton, debate, question

Topic  3
thank, trump, mr, mr trump, debate, republican, say, donald, right, donald trump

Topic  4
thank, states, united, united states, people, country, secretary, secretary clinton, america, percent

Topic  5
president, thank, obama, tax, years, jobs, america, economy, percent, taxes

Topic  6
states, say, united states, united, trump, tax, mr, mr trump, republican, taxes

Topic  7
right, thank, say, time, lot, debate, got, lets, ive, sanders

Topic  8
right, debate, president, republican, presidential, presidential debate, welcome, republican presidential, candidates, lets

Topic  9
right, world, isis, country, trump, middle, mr, mr trump, war, syria
